In [2]:
from src.data import MovingWindowProduct, MovingWindowSum

In [7]:
import torch
from dotmap import DotMap

In [8]:
# Config

device = "cuda" if torch.cuda.is_available() else "cpu"

config = {
'model':
  {
    'n_layer': 1,
    'n_head': 1,
    'n_embd': 256,
    'linear': True,
  },

'data':
  {
    'name': 'window',
    'min_num': 1,
    'max_num': 16,
    'k': 2,
    'p': 17,
    'sep': 17,
    'cot': False,
    'num_tokens': 16,
    'n_train': 256,
    'n_test': 64,
    'fixed_len': True,
  },

'train':
  {
    'lr': 0.0001,
    'grad_clip': -1,
    'num_steps': 500,
    'norm_type': "none_rank",
    'wandb': True,
    'save_ckpt': False,
    'ckpt_freq': 20,
  }
}
config = DotMap(config)

In [10]:
data_samplers = {}
data_samplers['mws'] = MovingWindowSum(
    min_num=config.data.min_num,
    max_num=config.data.max_num,
    k=config.data.k,
    p=config.data.p,
    sep = 17,
)
data_samplers['mwp'] = MovingWindowProduct(
    min_num=config.data.min_num,
    max_num=config.data.max_num,
    k=config.data.k,
    p=config.data.p,
    sep = 17,
)

In [ ]:
data_samplers['mws'].sample()

In [11]:
n_train, n_test, num_tokens = (
    config.data.n_train,
    config.data.n_test,
    config.data.num_tokens,
)

# --- MIXED BATCH SAMPLING ---
task_names = list(data_samplers.keys())
n_tasks = len(task_names)

n_train_each = n_train // n_tasks
n_test_each = n_test // n_tasks

mixed_train = []
mixed_test = []

for name, sampler in data_samplers.items():
    data = sampler.sample(
        num_samples=n_train_each + n_test_each,
        num_tokens=num_tokens,
    )
    train_part = data[:n_train_each, :]
    test_part = data[n_train_each:, :]
    mixed_train.append(train_part)
    mixed_test.append(test_part)

train_data = torch.cat(mixed_train, dim=0)
test_data = torch.cat(mixed_test, dim=0)

In [22]:
perm = torch.randperm(train_data.size(0))
train_data = train_data[perm]
perm = torch.randperm(test_data.size(0))
test_data = test_data[perm]

In [ ]:
config.